### Medical  Cost Prediction

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,RidgeCV,LassoCV,ElasticNetCV


In [ ]:
data=pd.read_csv("../input/insurance/insurance.csv")
data

In [ ]:
data.describe(include="all")

In [ ]:
data.info()

### Preprocesssing Step

In [ ]:
print("Total Number of Null Value :", data.isnull().sum().sum())

In [ ]:
data["children"] = data["children"].astype(str)

In [ ]:
print(data.select_dtypes("object") ) 

print(" ")
print(" ")

print(" 'Total Non-Numeric Column' :", len(data.select_dtypes("object").columns) )

In [ ]:
{column : list(data[column].unique()) for column in data.select_dtypes("object") }

In [ ]:
def binary_encode(df,column,positive_value):
    df=df.copy()
    df[column]=df[column].apply(lambda x: 1 if x==positive_value else 0)
    return df
    

def onehot_encode (df,column,prefix):
    df=df.copy()
    dummies=pd.get_dummies(df[column],prefix=prefix)
    df=pd.concat([df, dummies],axis=1)
    df=df.drop(column,axis=1)
    return df

In [ ]:
def preprocessing (df,scaler,train_size=0.7):
    df=df.copy()
    
    #binary encode for Sex and Smoker Column
    df=binary_encode(df,'sex', "male" )
    df=binary_encode(df,'smoker', "yes" )
    
    
    #onehot encoding for children and region
    df=onehot_encode(df,'children', "ch" )
    df=onehot_encode(df,'region', "re" )
    
    #spliting
    y=df["charges"].copy()
    X=df.drop("charges" ,axis=1).copy()
    
    
    #scaler
    X=pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
   
    #spliting train and test
    X_train,X_test,y_train,y_test =train_test_split(X,y,train_size=0.7,shuffle=True,random_state=123)
    
    return X_train,X_test,y_train,y_test

In [ ]:
X_train,X_test,y_train,y_test=preprocessing(data,StandardScaler())

In [ ]:
X_train,X_test,y_train,y_test

In [ ]:
### Training

In [ ]:
models ={
    "LinearRegression :":LinearRegression(),
    "Ridge            :":Ridge(),
    "Lasso            :":Lasso(),
    "ElasticNet       :":ElasticNet(),
    "RidgeCV          :":RidgeCV(),
    "LassoCV          :":LassoCV(),
    "ElasticNetCV     :":ElasticNetCV()
}

for model in models.values():
    model.fit(X_train,y_train)

In [ ]:
print ("Model R^2 Scores: \n------------------------")
for name, model in models.items():
    print(name,model.score(X_test,y_test))